In [1]:
import lightgbm as lgb
import pandas as pd
import catboost as cb
import xgboost as xgb

In [2]:
import catboost.datasets

epsilon = catboost.datasets.epsilon()

In [3]:
epsilon[1].head()

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,-1.0,0.005439,0.013331,-0.002186,-0.014590,0.015632,-0.032606,-0.004455,0.013611,0.024088,...,0.029023,0.028153,-0.001714,-0.048453,-0.030330,-0.006301,-0.022238,-0.009459,0.027544,-0.026216
1,1.0,0.001442,0.033820,-0.048836,0.000652,-0.028718,0.013421,-0.006827,0.053082,-0.016931,...,-0.016412,0.005543,-0.017588,0.005169,0.037386,-0.002206,0.023466,0.023459,0.036497,0.033899
2,1.0,0.004597,-0.042784,-0.004416,-0.005693,0.000731,-0.025873,0.031471,0.059522,0.003261,...,-0.020841,-0.030902,0.005387,-0.017727,-0.011851,0.007834,-0.002806,-0.004059,0.024565,-0.001050
3,-1.0,-0.009080,0.017605,-0.009870,0.007386,0.021338,-0.042682,-0.004471,0.035229,0.037935,...,-0.025915,0.005119,0.022496,-0.005170,-0.023424,-0.026319,-0.036478,-0.036575,-0.002760,-0.021496
4,1.0,-0.011689,-0.021413,0.012358,-0.012036,-0.009324,-0.023587,0.007309,0.034027,-0.020042,...,0.006731,0.011447,-0.015999,-0.016919,0.047534,-0.004458,0.013541,0.036077,-0.004313,0.008140


Load data
====

In [4]:
epsilon_test = epsilon[1] #pd.read_csv('test', header=None, sep='\t', nrows=100000)

In [19]:
X_test = epsilon_test.drop(0, axis=1).values
x_label = epsilon_test.iloc[:,0].map(lambda x: 1 if x > 0.5 else 0)
xgb_test = xgb.DMatrix(X_test, label=x_label)


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [20]:
params = {
    'nthread': 8,
    'tree_method': 'gpu_hist',
    'grow_policy ': 'lossguide',
    'eta': 0.01,
    'max_depth': 8,
    'objective': 'binary:logistic'
}

bst = xgb.train(params, xgb_test)

In [ ]:
epsilon_model = cb.CatBoost()
catboost_pool = cb.Pool(X_test)
print(epsilon_model.tree_count_)

8 thread apply
====

In [ ]:
#bst = xgb.Booster({'nthread': 32})  # set 8 thread in openmp threadpool
#bst.load_model('XGBoost.model')  # load data
#lgb_model = lgb.Booster(model_file='LightGBM_model.txt')

In [ ]:
%timeit -r 5 bst.predict(xgb_test, ntree_limit=8000)

In [ ]:
%timeit -r 5 lgb_model.predict(X_test, num_iteration=8000)

In [ ]:
%timeit -r 5 epsilon_model.predict(catboost_pool, thread_count=32, ntree_end=8000)

1 thread apply
====

In [ ]:
bst = xgb.Booster({'nthread': 1})  # set 1 thread in openmp threadpool
bst.load_model('XGBoost.model')  # load data
lgb_model = lgb.Booster(model_file='LightGBM_model.txt')

In [ ]:
%time _ = bst.predict(xgb_test, ntree_limit=8000)

In [ ]:
%time _ = lgb_model.predict(X_test, num_iteration=8000)

In [ ]:
%time _ = epsilon_model.predict(catboost_pool, thread_count=1, ntree_end=8000)

In [ ]:
import gc

In [ ]:
gc.collect()